In [ ]:
import json, time, pathlib
import matplotlib.pyplot as plt

from pprint import pprint

import sklearn.datasets
import sklearn.metrics

import autosklearn.classification

/home/joao/miniconda3/envs/as-env/lib/python3.9/site-packages/sklearn/utils/fixes.py:28: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version  # type: ignore


# Escolha do Dataset + Parâmetros Experimentos

In [ ]:
dataset_name = "iris"

X, y = sklearn.datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, random_state=1
)

limit = 60 * 1 # 1 minute

# Definição dos Experimentos

In [4]:
from multiprocessing import Pool
from AutoSklearn.without_ensemble import without_ensemble_experiment
from AutoSklearn.with_ensemble import with_ensemble_experiment

def handle_result(result):
    # Aqui você processa o resultado assim que ele chega,
    # mesmo que o outro experimento ainda esteja rodando.
    predictions = result.predict(X_test)
    print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))



with Pool(processes=2) as pool:
    # dispara sem bloquear, e registra uma função de callback
    pool.apply_async(
        without_ensemble_experiment,
        args=(X_train, y_train, limit, dataset_name),
        callback=handle_result
    )
    pool.apply_async(
        with_ensemble_experiment,
        args=(X_train, y_train, limit, dataset_name),
        callback=handle_result
    )

    # fecha o pool para novas tarefas, mas mantém vivo
    pool.close()
    # aqui o main vai “esperar” pelas callbacks, mas não bloqueia
    # ponto a ponto: handle_result roda assim que cada job acabar
    pool.join()


ModuleNotFoundError: No module named 'AutoSklearn'